Data taken from [the movies dataset in Kaggle](https://www.kaggle.com/rounakbanik/the-movies-dataset)

In [4]:
import pandas as pd
import os

In [5]:
CWD = os.getcwd()
DATA_DIR = os.path.join(CWD, 'data')

In [6]:
ratings_df = pd.read_csv(os.path.join(DATA_DIR, 'ratings.csv'))

In [7]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [8]:
ratings_df.shape

(26024289, 4)

# EDA

In [9]:
n_users = ratings_df['userId'].nunique()
print('Avg number of ratings per user: {}'.format(int(ratings_df.shape[0] / n_users)))

Avg number of ratings per user: 96


In [10]:
ratings_df.groupby('rating').size()

rating
0.5     404897
1.0     843310
1.5     403607
2.0    1762440
2.5    1255358
3.0    5256722
3.5    3116213
4.0    6998802
4.5    2170441
5.0    3812499
dtype: int64

In [11]:
movies_metadata_df = pd.read_csv(os.path.join(DATA_DIR, 'movies_metadata.csv'))

C:\Users\Kevin's PC\AppData\Local\Temp\ipykernel_19052\2140507666.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_metadata_df = pd.read_csv(os.path.join(DATA_DIR, 'movies_metadata.csv'))


In [12]:
movies_metadata_df.tail()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0
45465,False,NaN,0,[],NaN,461257,tt6980792,en,Queerama,50 years after decriminalisation of homosexual...,...,2017-06-09,0.0,75.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Queerama,False,0.0,0.0


In [13]:
movies_metadata_df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [14]:
movies_metadata_df.shape

(45466, 24)

In [15]:
ratings_df['movieId'].nunique()

45115

In [16]:
import re
def find_date(id):
    return bool(re.match('^[0-9]+$', id))

In [17]:
movies_metadata_df['keep_row'] = movies_metadata_df['id'].apply(find_date)

In [18]:
movies_metadata_df['keep_row'].value_counts()

True     45463
False        3
Name: keep_row, dtype: int64

In [19]:
movies_metadata_df[movies_metadata_df['keep_row'] == False]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,keep_row
19730,- Written by Ørnås,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'name': 'Carousel Productions', 'id': 11176}...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1997-08-20,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
29503,Rune Balot goes to a casino connected to the ...,1.931659,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...","[{'iso_3166_1': 'US', 'name': 'United States o...",2012-09-29,0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
35587,Avalanche Sharks tells the story of a bikini ...,2.185485,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"[{'name': 'Odyssey Media', 'id': 17161}, {'nam...","[{'iso_3166_1': 'CA', 'name': 'Canada'}]",2014-01-01,0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [20]:
movies_metadata_df = movies_metadata_df[movies_metadata_df['keep_row'] == True]
movies_metadata_df['keep_row'].value_counts()

True    45463
Name: keep_row, dtype: int64

In [21]:
movies_metadata_df['id'] = movies_metadata_df['id'].astype('int64')
movies_metadata_df.dtypes

adult                     object
belongs_to_collection     object
budget                    object
genres                    object
homepage                  object
id                         int64
imdb_id                   object
original_language         object
original_title            object
overview                  object
popularity                object
poster_path               object
production_companies      object
production_countries      object
release_date              object
revenue                  float64
runtime                  float64
spoken_languages          object
status                    object
tagline                   object
title                     object
video                     object
vote_average             float64
vote_count               float64
keep_row                    bool
dtype: object

In [22]:
data = ratings_df.merge(movies_metadata_df.add_suffix('_movies'), left_on = 'movieId', right_on = 'id_movies', how = 'inner')
data.shape

(11437637, 29)

## Highest Rated Movie on Avg.

In [23]:
data.columns

Index(['userId', 'movieId', 'rating', 'timestamp', 'adult_movies',
       'belongs_to_collection_movies', 'budget_movies', 'genres_movies',
       'homepage_movies', 'id_movies', 'imdb_id_movies',
       'original_language_movies', 'original_title_movies', 'overview_movies',
       'popularity_movies', 'poster_path_movies',
       'production_companies_movies', 'production_countries_movies',
       'release_date_movies', 'revenue_movies', 'runtime_movies',
       'spoken_languages_movies', 'status_movies', 'tagline_movies',
       'title_movies', 'video_movies', 'vote_average_movies',
       'vote_count_movies', 'keep_row_movies'],
      dtype='object')

In [24]:
avg_rating = data.groupby(by = 'movieId')['rating'].mean().reset_index(name = 'avg_rating').sort_values(by = 'avg_rating', ascending = False)
n_ratings = data.groupby(by = 'movieId').size().reset_index(name = 'num_ratings')

In [29]:
all_ratings = avg_rating.merge(n_ratings, on = 'movieId', how = 'inner')
all_ratings.head()

,movieId,avg_rating,num_ratings
0,95977,5.0,1
1,167666,5.0,1
2,130544,5.0,1
3,129530,5.0,1
4,164278,5.0,1


Make rating categories so we can see highest rated movies with most votes.

In [44]:
def rating_categories(avg_rating):
    if avg_rating >= 4.5:
        return '4.5+'
    elif avg_rating >= 4:
        return '4-4.5'
    elif avg_rating >= 3:
        return '3-4'
    elif avg_rating >= 2:
        return '2-3'
    else:
        return 'low'

In [45]:
all_ratings['rating_category'] = all_ratings['avg_rating'].apply(rating_categories)

In [46]:
all_ratings['rating_category'].value_counts()

3-4      4735
2-3      2179
low       342
4-4.5     264
4.5+       45
Name: rating_category, dtype: int64

In [48]:
top_10_each_category = all_ratings.sort_values(by = 'num_ratings', ascending = False).groupby(by = 'rating_category').head(10)

In [50]:
top_10_each_category[top_10_each_category['rating_category'] == '4.5+']

,movieId,avg_rating,num_ratings,rating_category
38,109441,4.50,4,4.5+
41,106113,4.50,3,4.5+
40,169496,4.50,2,4.5+
39,161094,4.50,2,4.5+
43,95578,4.50,2,4.5+
16,164777,4.75,2,4.5+
22,169022,4.50,2,4.5+
21,129360,4.50,2,4.5+
28,101538,4.50,2,4.5+
0,95977,5.00,1,4.5+


In [57]:
movies_metadata_df.merge(top_10_each_category[top_10_each_category['rating_category'] == '4.5+'], left_on = 'id', right_on = 'movieId', how = 'inner')[['original_title', 'num_ratings','avg_rating']].sort_values(by = 'num_ratings', ascending = False)

,original_title,num_ratings,avg_rating
3,Don't Go Near the Water,4,4.50
6,Don't Tell the Wife,3,4.50
1,Hoop Reality,2,4.50
2,친구사이?,2,4.50
4,Evel Knievel,2,4.50
5,"Usain Bolt, La Légende",2,4.50
7,Kitchen,2,4.50
8,Il Magnifico Cornuto,2,4.75
9,The Gnomes' Great Adventure,2,4.50
0,The Man Behind The Gun,1,5.00


Looks like this dataset doesn't have many high rated movies. Let's see the next highest rating category.

In [58]:
movies_metadata_df.merge(top_10_each_category[top_10_each_category['rating_category'] == '4-4.5'], left_on = 'id', right_on = 'movieId', how = 'inner')[['original_title', 'num_ratings','avg_rating']].sort_values(by = 'num_ratings', ascending = False)

,original_title,num_ratings,avg_rating
5,The Million Dollar Hotel,91082,4.429015
8,Terminator 3: Rise of the Machines,87901,4.169975
4,Солярис,84078,4.152246
3,The 39 Steps,77045,4.132299
0,Once Were Warriors,67662,4.266531
1,Trois couleurs : Rouge,66512,4.016057
9,License to Wed,60024,4.230716
2,Sleepless in Seattle,57070,4.339811
6,5 Card Stud,56827,4.104167
7,Men in Black II,52474,4.105347


## Plan for Recommendation System

I would like to build a recommendation system that takes the ratings a user has made, and matches 10 movies that are most similar to the highest rated movies the user rated.
We will use the overview and genres of the movie to define *similarity*.

In [100]:
# clean genres column to be a list of genres seperated by space
import json

def extract_genre(genre):
    """
    use regex to find each dictionary in string,
    convert string to dictionary
    extract name
    return genres as single string
    """
    genre_str = genre.replace('[','').replace(']','').replace('\'', '\"')
    genre_list = []
    # loop through each dict in string using regex
    for small_dict in re.findall('\{.*?\}', genre_str):
        item_dict = json.loads(small_dict)
        genre_list.append(item_dict['name'])
    return ' '.join(genre_list)

In [98]:
movies_metadata_df['genres_cleaned'] = movies_metadata_df['genres'].apply(extract_genre)

In [99]:
movies_metadata_df[['genres_cleaned', 'genres']].iloc[0]

genres_cleaned                              Animation Comedy Family
genres            [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
Name: 0, dtype: object

In [110]:
def append_genres(df):
    return ' '.join([str(df['overview']), str(df['genres_cleaned'])])

In [111]:
# append genre to end of overview
movies_metadata_df['overview_final'] = movies_metadata_df.apply(append_genres, axis = 1)

In [115]:
movies_metadata_df['overview_final'].values[400]

"Starts off in the 15th century, with Connor McLeod (Christopher Lambert) training with another immortal swordsman, the Japanese sorcerer Nakano (Mako). When an evil immortal named Kane (Mario Van Peebles) kills the old wizard, the resulting battle leaves him buried in an underground cave. When Kane resurfaces in the 20th century to create havoc, it's up to McLeod to stop him. Action Fantasy Science Fiction"